# 1번문제

# 2번문제

In [44]:
test_input = np.random.rand(1,3,128,128)

In [43]:
test_filter = np.random.rand(1,3,20,20)

In [40]:
test.shape

(1, 1, 128, 128)

In [47]:
import numpy as np

def im2col(input_data, filter_h, filter_w, stride=1):
    N, C, H, W = input_data.shape
    out_h = (H - filter_h)//stride + 1
    out_w = (W - filter_w)//stride + 1

    col = np.zeros((N, C, filter_h, filter_w, out_h, out_w))

    for y in range(filter_h):
        y_max = y + stride*out_h
        for x in range(filter_w):
            x_max = x + stride*out_w
            col[:, :, y, x, :, :] = input_data[:, :, y:y_max:stride, x:x_max:stride]

    col = col.transpose(0, 4, 5, 1, 2, 3).reshape(N*out_h*out_w, -1)
    return col

def conv_forward(input, filter, stride=1):
    FN, C, FH, FW = filter.shape
    N, C, H, W = input.shape
    out_h = 1 + int((H - FH) / stride)
    out_w = 1 + int((W - FW) / stride)

    col = im2col(input, FH, FW, stride)
    col_W = filter.reshape(FN, -1).T

    
    outputs = np.dot(col, col_W)
    outputs = outputs.reshape(N, out_h, out_w, -1).transpose(0, 3, 1, 2)
    
    return outputs

In [51]:
test_output = conv_forward(test_input,test_filter)

(1, 1, 109, 109)


In [73]:
def col2im(col, input_shape, filter_h, filter_w, stride=1):
    N, C, H, W = input_shape
    out_h = (H - filter_h)//stride + 1
    out_w = (W - filter_w)//stride + 1
    col = col.reshape(N, out_h, out_w, C, filter_h, filter_w).transpose(0, 3, 4, 5, 1, 2)

    img = np.zeros((N, C, H + stride - 1, W + stride - 1))
    for y in range(filter_h):
        y_max = y + stride*out_h
        for x in range(filter_w):
            x_max = x + stride*out_w
            img[:, :, y:y_max:stride, x:x_max:stride] += col[:, :, y, x, :, :]

    return img[:, :, H, W]

def conv_backward(grad_prev_outputs, inputs, filter, stride=1):
    FN, C, FH, FW = filter.shape
    N, C, H, W = inputs.shape
    col = im2col(inputs, FH, FW, stride)
    col_W = filter.reshape(FN, -1).T
    
    grad_prev_outputs = grad_prev_outputs.transpose(0,2,3,1).reshape(-1, FN)

    grad_filter = np.dot(col.T, grad_prev_outputs)
    grad_filter = grad_filter.transpose(1, 0).reshape(FN, C, FH, FW)


    dcol = np.dot(grad_prev_outputs, col_W.T)
    grad_outputs = col2im(dcol, inputs.shape, FH, FW, stride)

    return grad_outputs, grad_filter

In [74]:
conv_backward(test_output, test_input, test_filter)

IndexError: index 128 is out of bounds for axis 2 with size 128

# 3번문제

In [1]:
class Convolution:
    def __init__(self, W, b, stride=1, pad=0):
        self.W = W
        self.b = b
        self.stride = stride
        self.pad = pad
        
        # 中間データ（backward時に使用）
        self.x = None   
        self.col = None
        self.col_W = None
        
        # 重み・バイアスパラメータの勾配
        self.dW = None
        self.db = None

    def forward(self, x):
        FN, C, FH, FW = self.W.shape
        N, C, H, W = x.shape
        out_h = 1 + int((H + 2*self.pad - FH) / self.stride)
        out_w = 1 + int((W + 2*self.pad - FW) / self.stride)

        col = im2col(x, FH, FW, self.stride, self.pad)
        col_W = self.W.reshape(FN, -1).T

        out = np.dot(col, col_W) + self.b
        out = out.reshape(N, out_h, out_w, -1).transpose(0, 3, 1, 2)

        self.x = x
        self.col = col
        self.col_W = col_W

        return out

    def backward(self, dout):
        FN, C, FH, FW = self.W.shape
        dout = dout.transpose(0,2,3,1).reshape(-1, FN)

        self.db = np.sum(dout, axis=0)
        self.dW = np.dot(self.col.T, dout)
        self.dW = self.dW.transpose(1, 0).reshape(FN, C, FH, FW)

        dcol = np.dot(dout, self.col_W.T)
        dx = col2im(dcol, self.x.shape, FH, FW, self.stride, self.pad)

        return dx